In [ ]:
import os
import numpy as np
import random
import math
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as Data
import scipy.io
import glob
import torch.nn.functional as F
from numpy import load


In [ ]:
BATCH_SIZE = 10#100
BATCH_SIZEV = 13#560
BATCH_SIZEV2 = 32

# longest epoch len in the validation set
len_val = 1150
EPOCH = 1000
h = 600
w = 2
in_w = 2
sqe = 1200 # can try 5
cls = 4
crop = 600#1240

In [ ]:
class Encode_micro(nn.Module):
    def __init__(self):
        super(Encode_micro, self).__init__()
        
        self.rnn = nn.GRU(2, 32, 2, bidirectional = True, dropout=0.5, batch_first = True)
        
        self.fc = nn.Sequential( 
            nn.Linear(32*2, 128),
        )
     
    def forward(self, src):
        
        #src = [src sent len, batch size]
        outputs, hidden = self.rnn(src)
        hidden = self.fc(outputs[:,-1,:])
        return outputs, hidden


class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.rnn = nn.GRU(2, 64, 3, bidirectional = True, dropout=0.5, batch_first = True)
        
        self.cnn = nn.Sequential( 
        nn.Conv1d(2, 128, 5, stride=1, padding=2),
        nn.LeakyReLU(inplace=False),
        nn.MaxPool1d(6, stride=6),#60
        nn.Conv1d(128, 128, 5, stride=1, padding=2),
        nn.LeakyReLU(inplace=False),
        #nn.Dropout(0.5),
        #nn.MaxPool1d(3, stride=3),#20
        nn.Conv1d(128, 128, 5, stride=1, padding=2),
        nn.LeakyReLU(inplace=False),
        nn.MaxPool1d(5, stride=5),#10
        nn.Conv1d(128, 128, 5, stride=1, padding=2),
        nn.LeakyReLU(inplace=False),
        # #nn.MaxPool1d(2, stride=2),#5
        nn.Conv1d(128, 128, 5, stride=1, padding=2),
        # nn.LeakyReLU(inplace=False),
        # #nn.MaxPool1d(2, stride=2),#2
        # nn.Conv1d(128, 256, 9, stride=1, padding=4),     
        nn.Dropout(0.5),
        #nn.LeakyReLU(inplace=False), #put it back 2020 706
        )
         
        self.cnn_ACT = nn.Sequential( 
        nn.Conv1d(5, 16, 9, stride=1, padding=4),
        nn.LeakyReLU(inplace=False),
        nn.Conv1d(16, 32, 9, stride=1, padding=4),
        nn.LeakyReLU(inplace=False),
        nn.Conv1d(32, 64, 9, stride=1, padding=4),
        nn.Dropout(0.5),
        )
        self.cnn3 = nn.Sequential( 
        nn.Conv1d(128+64+128, 256, 11, stride=1, padding=5),
        nn.LeakyReLU(inplace=False),
        nn.Conv1d(256, 256, 15, stride=1, padding=7),
        nn.LeakyReLU(inplace=False),
        nn.Conv1d(256, 512, 21, stride=1, padding=10),
        nn.Dropout(0.5),
        )

    def forward(self, src, time):
        
        #src = [src sent len, batch size]
        src1 = src.permute(0, 2, 1)
        src11 = time.permute(0, 2, 1)
        #print(src1.size())
        src2 = self.cnn(src1)
        outputs, hidden = self.rnn(src)
        outputs = outputs[:,::30,:]
        src22 = outputs.permute(0, 2, 1)
        src222 = self.cnn_ACT(src11)
        #print src2.size()
        #outputs2 = outputs.permute(0, 2, 1)
        #src3= torch.cat((src11 src2),1)
        src3= torch.cat((src2, src22, src222),1)
        src4 = self.cnn3(src3)
        src4= torch.cat((src4, src11[:,:,:]),1)
        src5 = src4.permute(0, 2, 1)
        

        return src5
    
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super(Attention, self).__init__()
        
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        
        self.attn = nn.Linear((enc_hid_dim * 1) + dec_hid_dim, dec_hid_dim, bias=True)
        self.v = nn.Parameter(torch.rand(dec_hid_dim))
        
    def forward(self, hidden, encoder_outputs):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src sent len, batch size, enc hid dim * 2]
        

        batch_size = encoder_outputs.shape[0]
        src_len = encoder_outputs.shape[1]
        
        #repeat encoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        

        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        energy = energy.permute(0, 2, 1)
        
        v = self.v.repeat(batch_size, 1).unsqueeze(1)
        

        attention = torch.bmm(v, energy).squeeze(1)
        
        return F.softmax(attention, dim=1)



class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super(Encoder, self).__init__()
        
        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.dropout = dropout
        
        self.rnn = nn.GRU(512+5, enc_hid_dim, 1, bidirectional = False, dropout=0.2, batch_first = True)
        
        self.fc = nn.Sequential( 
            nn.Linear(enc_hid_dim * 1, dec_hid_dim, bias=True),
        )
     
    def forward(self, src):
        
        #src = [src sent len, batch size]
        


        outputs, hidden = self.rnn(src)


        hidden = self.fc(hidden[-1,:,:])

        return outputs, hidden


class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super(Decoder, self).__init__()

        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.output_dim = output_dim
        self.attention = attention
        self.rnn = nn.GRU((enc_hid_dim * 1 ) + emb_dim, dec_hid_dim,1, bidirectional = False, batch_first = True, dropout=0.2)
        self.out = nn.Sequential( 

            nn.Linear((enc_hid_dim * 1) + dec_hid_dim + emb_dim + 512+5 + 128, output_dim),
            #nn.Dropout(0.5),
        )
        
        self.dropout = nn.Dropout(dropout)


    def forward(self, input, hidden, encoder_outputs, cnnf, micro):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src sent len, batch size, enc hid dim * 2]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        
        a = self.attention(hidden, encoder_outputs)
        #a = [batch size, src len]
        a = a.unsqueeze(1)
        #a = [batch size, 1, src len]
        encoder_outputs = encoder_outputs.permute(0, 1, 2)
        
        #encoder_outputs = [batch size, src sent len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        

        rnn_input = torch.cat((input, weighted), dim = 2)

        rnn_input = rnn_input.permute(1, 0, 2)

        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))

        output = output.squeeze(1)
        output = output.unsqueeze(0)
        cnnf = cnnf.unsqueeze(0)
        micro=micro.unsqueeze(0)

        output = self.out(torch.cat((output, weighted, input, cnnf, micro), dim = 2))

        return output, hidden.squeeze(0)


class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device = 0):
        super(Seq2Seq, self).__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        self.rnn = Encode_micro()
        print(self.device)

        
    def forward(self, src, src2, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src sent len, batch size]
        #trg = [trg sent len, batch size]
        batch_size = trg.shape[0]
        max_len = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(batch_size, max_len, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        encoder_outputs, hidden = self.encoder(src)

        input = trg[:,0]


        

        
        for t in range(0, max_len):
            out, hid = self.rnn(src2[:,t*30:(t+1)*30,:])
            output, hidden = self.decoder(input, hidden, encoder_outputs, src[:,t,:],hid)
            if output.size()[0] == 1:
                outputs[:,t,:] = output[:,:]
            else:
                outputs[:,t,:] = output[:,0,:]
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            input = trg[:,t] if teacher_force else output.squeeze(0)
        

        return outputs


In [ ]:

class features_dataset(Data.Dataset):

    def __init__(self, mode, train):

        self.mode = mode
        
        
        
        

        if train == 0:
            #training path
            self.night  = glob.glob('/media/godzilla/database/Data8/CVdata/T3/*.mat')


        else:
            #testing path
            self.night  = glob.glob('/media/godzilla/database/Data8/CVdata/V3/*.mat')




            
        print(len(self.night))
    def __getitem__(self, index):


        
        #training result: only check the first 600 epochs
        if self.mode == 0:
            
            data = scipy.io.loadmat(self.night[index])
            x = np.squeeze(data['hr'])/60.0
            xms = np.squeeze(data['hr2'])/60.0
            x2 = np.squeeze(data['act'])
            x3 = np.squeeze(data['clock'])
            x4 = np.squeeze(data['time'])
            x5 = np.squeeze(data['hz'])
            # act can be normalized into 0 to 1 like HR
            x = (x-x.mean())/x.std()
            x2 = (x2-x2.mean())/x2.std()
            #x = (x-x.min())/(x.max()-x.min())
            #x2 = (x2-x2.min())/(x2.max()-x2.min())
            y = np.squeeze(data['stg_cor_cln'])
            # y0 = np.squeeze(data['stg'])
            #y = np.where(y0 == 5, 5, y)
            # y = np.where(y0 == 0, 0, y)
            # y = np.where(y0 == 4, 4, y)
            # y = np.where(y0 == 3, 3, y)
            # y = np.where(y0 == 1, 1, y)
            x = np.expand_dims(np.squeeze(x), axis=1)
            x2 = np.expand_dims(np.squeeze(x2), axis=1)
            x3 = np.expand_dims(np.squeeze(x3), axis=1)
            x4 = np.expand_dims(np.squeeze(x4), axis=1)
            x5 = np.expand_dims(np.squeeze(x5), axis=1)
            y = np.expand_dims(np.squeeze(y), axis=1)

            
            x_final = np.zeros((600*30,2))
            time = np.zeros((600,5))
            y_final = np.zeros((600,1))
            z_mask = np.zeros((600,1))
            z_tmp = y
            z_tmp = np.where(z_tmp < 5, 1, z_tmp)
            z_tmp = np.where(z_tmp == 5, 0, z_tmp)
             
            #y = np.where(y == 1, 0, y)
            y = np.where(y == 2, 1, y)
            y = np.where(y == 3, 2, y)
            y = np.where(y == 4, 3, y)
            y = np.where(y == 5, 0, y)
            #x_final[10*15:,:] = x[:600*15,:]
            #y_final[10:,:] = y[:600,:]


            s = y.shape[0]


            if s >= 600:
                x_final[:,0] = x[:600*30,0]
                x_final[:,1] = x2[:600*30,0]
                y_final[:,:] = y[:600,:]
                z_mask[:,:] = z_tmp[:600,:]
                time[:,0] = x5[:600,0]
                time[:,1] = x3[:600,0]
                time[:,2] = x4[:600,0]
                time[:,3] = xms[0,:600]
                time[:,4] = xms[1,:600]
            else:
                x_final[:x.shape[0],0] = x[:x.shape[0],0]
                x_final[:x2.shape[0],1] = x2[:x2.shape[0],0]
                y_final[:y.shape[0],:] = y[:y.shape[0]:]
                z_mask[:y.shape[0],:] = z_tmp[:y.shape[0],:]
                time[:y.shape[0],0] = x5[:y.shape[0],0]
                time[:y.shape[0],1] = x3[:y.shape[0],0]
                time[:y.shape[0],2] = x4[:y.shape[0],0]
                time[:y.shape[0],3] = xms[0,:y.shape[0]]
                time[:y.shape[0],4] = xms[1,:y.shape[0]]           
        # random crop 600 epochs for training
        elif self.mode == 1:


            
            data = scipy.io.loadmat(self.night[index])
            x = np.squeeze(data['hr'])/60.0
            xms = np.squeeze(data['hr2'])/60.0
            x = (x-x.mean())/x.std()
            #x = (x-x.min())/(x.max()-x.min())
            
            x2 = np.squeeze(data['act'])
            x3 = np.squeeze(data['clock'])
            x4 = np.squeeze(data['time'])
            x5 = np.squeeze(data['hz'])
            #x2 = x2/x2.max()
            x2 = (x2-x2.mean())/x2.std()
            #x2 = (x2-x2.min())/(x2.max()-x2.min())
            #x2 = (x2-x2.mean())/x2.std()

            # act can be normalized into 0 to 1 like HR

            y = np.squeeze(data['stg_cor_cln'])
            #y0 = np.squeeze(data['stg'])
            # y = np.where(y0 == 5, 5, y)
            # y = np.where(y0 == 0, 0, y)
            # y = np.where(y0 == 4, 4, y)
            # y = np.where(y0 == 3, 3, y)
            # y = np.where(y0 == 1, 1, y)
            #print(y.shape)
            
            #print(x.shape)
            #print(y.shape)
            x = np.expand_dims(np.squeeze(x), axis=1)
            x2 = np.expand_dims(np.squeeze(x2), axis=1)
            x3 = np.expand_dims(np.squeeze(x3), axis=1)
            x4 = np.expand_dims(np.squeeze(x4), axis=1)
            x5 = np.expand_dims(np.squeeze(x5), axis=1)
            y = np.expand_dims(np.squeeze(y), axis=1)
            z_tmp = y
            z_tmp = np.where(z_tmp < 5, 1, z_tmp)
            z_tmp = np.where(z_tmp == 5, 0, z_tmp)
            #y = np.where(y == 1, 0, y)
            y = np.where(y == 2, 1, y)
            y = np.where(y == 3, 2, y)
            y = np.where(y == 4, 3, y)            
            y = np.where(y == 5, 0, y)
            s = y.shape[0]
            
            time = np.zeros((1200,5))
            #time[:,0] = np.arange(0, 1200, 1, dtype=float)/1200.0
            
            if s > 1200:
            
                x_final = np.zeros((1200*30,2))
                y_final = np.zeros(((1200),1))
                x_final[:,0] = x[:(1200)*30,0]
                x_final[:,1] = x2[:(1200)*30,0]
                y_final[:,:] = y[:1200,:]
                time[:,0] = x5[:1200,0]
                time[:,1] = x3[:1200,0]
                time[:,2] = x4[:1200,0]                         
                time[:,3] = xms[0,:1200]
                time[:,4] = xms[1,:1200]
                z_mask = z_tmp[:1200,:]   
                
            else:
                x_final = np.zeros(((1200)*30,2))
                y_final = np.zeros(((1200),1))
                z_mask = np.ones(((1200),1))
                #print(x.shape[0])
                x_final[:x.shape[0],0] = x[:x.shape[0],0]
                x_final[:x2.shape[0],1] = x2[:x2.shape[0],0]
                y_final[:y.shape[0],:] = y[:y.shape[0],:]
                z_mask[:y.shape[0],:] = z_tmp[:y.shape[0],:]
                
                time[:y.shape[0],0] = x5[:y.shape[0],0]
                time[:y.shape[0],1] = x3[:y.shape[0],0]
                time[:y.shape[0],2] = x4[:y.shape[0],0] 
                time[:y.shape[0],3] = xms[0,:y.shape[0]]
                time[:y.shape[0],4] = xms[1,:y.shape[0]]

            mask_ind = list(range(1200))
            random.shuffle(mask_ind)

            #z_mask[mask_ind[:1000],:] = 0

            # slen_tmp = [50, 100]
            # os_ind = random.randint(0, 1)
            # os_r = 1200-slen_tmp[os_ind]
            # os = random.randint(0, os_r-1)
            # z_mask[os:os+slen_tmp[os_ind],:] = 0

        #validation
        else:
            data = scipy.io.loadmat(self.night[index])
            x = np.squeeze(data['hr'])/60.0
            xms = np.squeeze(data['hr2'])/60.0
            x = (x-x.mean())/x.std()
            #x2 = np.squeeze(data['act'])
            #x2 = (x2-x2.mean())/x2.std()
            x2 = np.squeeze(data['act'])
            x3 = np.squeeze(data['clock'])
            x4 = np.squeeze(data['time'])
            x5 = np.squeeze(data['hz'])
            #x2 = x2/x2.max()
            x2 = (x2-x2.mean())/x2.std()



            #x = (x-x.min())/(x.max()-x.min())
            #x2 = (x2-x2.min())/(x2.max()-x2.min())            
            y = np.squeeze(data['stg_cor_cln'])
            #y0 = np.squeeze(data['stg'])
            # y = np.where(y0 == 5, 5, y)           
            # y = np.where(y0 == 0, 0, y)
            # y = np.where(y0 == 4, 4, y)
            # y = np.where(y0 == 3, 3, y)
            # y = np.where(y0 == 1, 1, y)
            x = np.expand_dims(np.squeeze(x), axis=1)
            x2 = np.expand_dims(np.squeeze(x2), axis=1)
            x3 = np.expand_dims(np.squeeze(x3), axis=1)
            x4 = np.expand_dims(np.squeeze(x4), axis=1)
            x5 = np.expand_dims(np.squeeze(x5), axis=1)
            y = np.expand_dims(np.squeeze(y), axis=1)
            
            
            z_tmp = y
            z_tmp = np.where(z_tmp < 5, 1, z_tmp)
            z_tmp = np.where(z_tmp == 5, 0, z_tmp)
            #y = np.where(y == 1, 0, y)
            y = np.where(y == 2, 1, y)
            y = np.where(y == 3, 2, y)
            y = np.where(y == 4, 3, y)
            y = np.where(y == 5, 0, y)
            tem_x = np.zeros((1200*30,2))
            tem_y = np.zeros((1200,1))
            tem_z = np.zeros((1200,1))
            tem_x[:x.shape[0],0] = x[:x.shape[0],0]
            tem_x[:x2.shape[0],1] = x2[:x2.shape[0],0]
            tem_y[:y.shape[0],:] = y[:,:]
            tem_z[:y.shape[0],:] = z_tmp[:,:]
            x1 = torch.FloatTensor(tem_x) 
            y1 = torch.LongTensor(tem_y)
            z1 = torch.LongTensor(tem_z)
            y11 = torch.LongTensor(np.squeeze(tem_y))
            y2 = torch.nn.functional.one_hot(y11, num_classes=4)


            time = np.zeros((1200,5))
            time[:y.shape[0],0] = x5[:y.shape[0],0]
            time[:y.shape[0],1] = x3[:y.shape[0],0]
            time[:y.shape[0],2] = x4[:y.shape[0],0]  
            time[:y.shape[0],3] = xms[0,:y.shape[0]]
            time[:y.shape[0],4] = xms[1,:y.shape[0]]
            #time[:,0] = np.arange(0, 1200, 1, dtype=float)/1200.0

            
            time = torch.FloatTensor(time)
            return x1, y1, y2, y.shape[0], z1, time
        #x = self.input_x[index].reshape((h, w))
        #y = self.input_y[index]			
        #x = x[:,[0, 1, 6]]	
        #x = x[:,:]
        x1 = torch.FloatTensor(x_final) 
        y1 = torch.LongTensor(y_final)
        z1 = torch.LongTensor(z_mask)
        y11 = torch.LongTensor(np.squeeze(y_final)) 
        time = torch.FloatTensor(time)

        #print(y1.size(), "                    "  ,x1.size())

        #print y1.size()
        #print(y1.size())
        
        #y2 = torch.zeros(y1.size()[0],4).scatter_(dim=1, index=y1, src=torch.tensor(1.0))
        y2 = torch.nn.functional.one_hot(y11, num_classes=4)
        #print y2.size()
        #quit()
        #print(y2.size())
        return x1, y1, y2, z1, time
    def __len__(self):
        if self.mode == 1:
            return(len(self.night))
        else:
            return(len(self.night))
            #return(len(self.x))
    def getName(self):
        return self.classes

In [ ]:
train_loader = torch.utils.data.DataLoader(
    features_dataset(mode = 1, train = 0),
    batch_size = BATCH_SIZE,
    shuffle=True,
    num_workers=4,
    drop_last=True
)
train_loader2 = torch.utils.data.DataLoader(
    features_dataset(mode = 0, train = 0),
    batch_size = BATCH_SIZEV,
    shuffle=False,
    num_workers=4,
    drop_last=True
)
val_loader = torch.utils.data.DataLoader(
    features_dataset(mode = 2, train = 1),
    batch_size = BATCH_SIZEV2,
    shuffle=False,
    num_workers=4,
    drop_last=True
)

In [ ]:
random.random() < 0.0

In [ ]:
INPUT_DIM = in_w
OUTPUT_DIM = 4
ENC_EMB_DIM = in_w
DEC_EMB_DIM = 4
#HID_DIM = 1
#N_LAYERS = 1
ENC_DROPOUT = 0.0
DEC_DROPOUT = 0.0
device = "cuda:0"


ENC_HID_DIM = 96

DEC_HID_DIM = 96

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)


cnnmodel = CNN().to(device)
model = Seq2Seq(enc, dec, device).to(device)


model = nn.DataParallel(model, device_ids=[0, 1])
cnnmodel = nn.DataParallel(cnnmodel, device_ids=[0, 1])

print(cnnmodel)
print(model)


MAX = 0.0


In [ ]:
class RCCLoss(nn.Module):
    def __init__(self, fn_weights=None, fp_weights=None):
        super(RCCLoss,self).__init__()
        #anti_eye = np.ones((k,k)) - np.eye(k)
        self.full_fn_weights = torch.Tensor(fn_weights)
        self.full_fp_weights = torch.Tensor(fp_weights)
        
    def forward(self, x, y):
        #print(x.size()[0])
        #print(y.size())
        #print(x)
        x = F.softmax(x, dim=1)
        yi = torch.cuda.LongTensor(y.unsqueeze(1))
        #print(yi)
        y1 = torch.zeros(x.size()[0],cls).cuda().scatter_(1, yi, 1)
        #print(y1)
        if y1.type() != x.data.type():
            y1 = y1.type_as(x.data)
        eps=1e-5
        
        x = torch.clamp(x, eps, 1. - eps) 
        
        logs = torch.log(x)
        
        logs_1_sub = torch.log(1.-x) # shape (m, k), dense. 0 is good. 
        #logs =  F.log_softmax(x)
        
        #logs_1_sub =  F.log_softmax(1.-x) # shape (m, k), dense. 0 is good. 
        #print(logs_1_sub)
        if self.full_fn_weights.type() != x.data.type():
            self.full_fn_weights = self.full_fn_weights.type_as(x.data)

        if self.full_fp_weights.type() != x.data.type():
            self.full_fp_weights = self.full_fp_weights.type_as(x.data)
        
        m_full_fn_weights = torch.mm(y1, self.full_fn_weights) # (m,k) . (k, k)
        
        m_full_fp_weights = torch.mm(y1, self.full_fp_weights) # (m,k) . (k, k)
        #print(y1.type())
        #print(self.full_fn_weights.type())
        #print(m_full_fp_weights.type())
        loss = (0.25*m_full_fn_weights * logs + 0.5*m_full_fp_weights * logs_1_sub)
        
        
        return loss

In [ ]:
w = 16433.0
l = 63668.0
d = 25685.0
r = 32015.0
aa = w + l + d+ r
weights = [1.0/0.0878, 1.0/0.4636, 1.0/0.1940, 1.0/0.2548]
fn_weights =[
 [1.0/0.0878, 0.00, 0.00, 0.00],
 [0.00, 1.0/0.4636, 0.00, 0.00],
 [0.00, 0.00, 1.0/0.1940, 0.00],
 [0.00, 0.00, 0.00, 1.0/0.2548]]
fp_weights4= [
 [0.00, (w/l)**0.5, (w/d)**0.5, (w/r)**0.5],
 [(l/w)**0.5 * 0.5, 0.00, (l/d)**0.5 *0.5, (l/r)**0.5 ],
 [(d/w)**0.5, (d/l)**0.5 * 0.5, 0.00, (r/l)**0.5 * 0.5],
 [(r/w)**0.5 * 0.5, (r/l)**0.5, (r/d)**0.5, 0.00]]

loss_func = RCCLoss(fn_weights=fn_weights, fp_weights=fp_weights4)
loss_func_CE = torch.nn.CrossEntropyLoss(reduction='none')

In [ ]:
# 4 class : no jitter N800 CE ICF

In [ ]:
loss_t = np.zeros(EPOCH)
loss_v = np.zeros(EPOCH)
acc_t = np.zeros(EPOCH)
acc_v = np.zeros(EPOCH)
for epoch in range(EPOCH):



    LR = 0.0001
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    optimizer2 = torch.optim.Adam(cnnmodel.parameters(), lr=LR)
    model.train()
    cnnmodel.train()

    #training

    for step, (x, y, y2, z, time) in enumerate(train_loader):
        
        
        y3 = y2
        sqe = 1200

        

        data_in = torch.FloatTensor(BATCH_SIZE, sqe*30, in_w)
        data_in2 = torch.FloatTensor(BATCH_SIZE, sqe, 5)
        data_out = torch.LongTensor(BATCH_SIZE, 1,1)
        data_out3 = torch.LongTensor(BATCH_SIZE, sqe, 1)
        data_mask = torch.LongTensor(BATCH_SIZE, sqe, 1)
        target = torch.FloatTensor(BATCH_SIZE, sqe, cls)

        data_in[:,:,:] = x[:,:sqe*30,:]
        data_out3[:,:,:] = y[:,:sqe,:]
        data_mask[:,:,:] = z[:,:sqe,:]
        data_in2[:,:,:] = time[:,:sqe,:]
        
        data_out4 = data_out3.squeeze(2).reshape(BATCH_SIZE*(sqe))
        data_mask = data_mask.squeeze(2).reshape(BATCH_SIZE*(sqe))
        data_out5 = data_out3.squeeze(2).reshape(BATCH_SIZE,(sqe)).permute(1,0)

        target[:,:,:] = y3[:,:sqe,:] 
        t_x = Variable(data_in.to(device))
        t_x2 = Variable(data_in2.to(device))


        t_y2 = Variable(data_out4.to(device))
        t_mask = Variable(data_mask.to(device))
        t_y3 = Variable(data_out5.to(device))
        trg = Variable(target.to(device))
        

        tmp = cnnmodel(t_x, t_x2)
        

        outputs = model(tmp, t_x, trg, 0.0)

        _, pred = torch.max(outputs[:,:,:].contiguous().view(-1,cls).data, 1)

        loss = loss_func_CE(outputs.contiguous().view(-1,cls), t_y2)
        total_loss_CE = torch.sum(loss * t_mask)
        loss = loss_func(outputs.contiguous().view(-1,cls), t_y2)
        t_mask2 = t_mask.unsqueeze(1).repeat(1,4)
        total_loss_RWL = -torch.sum(loss * t_mask2)

        total_loss = total_loss_CE + 0.01 * total_loss_RWL

        optimizer.zero_grad()   # clear gradients for next train
        optimizer2.zero_grad()

        total_loss.backward() 

        optimizer.step()        # apply gradients
        optimizer2.step()



    
    print("Training done")
    model.eval()
    cnnmodel.eval()
    if (epoch)%10 == 0:

        mse = np.zeros(BATCH_SIZEV)
        pmse = np.zeros(BATCH_SIZEV)
        count = 0.0
        acc = np.zeros(cls)
        total = np.zeros(cls)
        cf = np.zeros((cls,cls))

        for step, (x, y, y2, z, time) in enumerate(train_loader2):
            sqe800 = 600
            with torch.no_grad():

                y3 = y2

    
                #ii = 
                data_in = torch.FloatTensor(BATCH_SIZEV, sqe800*30, in_w)
                data_in2 = torch.FloatTensor(BATCH_SIZEV, sqe800, 5)
                data_out = torch.LongTensor(BATCH_SIZEV, 1,1)
                #data_out2 = torch.LongTensor(BATCH_SIZE * sqe)
                #data_out3 = torch.LongTensor(BATCH_SIZE, sqe+1, 1)
                data_out3 = torch.LongTensor(BATCH_SIZEV, sqe800, 1)
                data_mask = torch.LongTensor(BATCH_SIZEV, sqe800, 1)
                target = torch.FloatTensor(BATCH_SIZEV, sqe800, cls)
                #target2 = torch.FloatTensor(BATCH_SIZE, sqe-1, cls)
        
                data_in[:,:,:] = x[:,:sqe800*30,:]
                data_in2[:,:,:] = time[:,:sqe800,:5]

                data_out3[:,:,:] = y[:,:sqe800,:]
                data_mask[:,:,:] = z[:,:sqe800,:]
                
                data_out4 = data_out3.squeeze(2).reshape(BATCH_SIZEV*(sqe800))
                data_mask = data_mask.squeeze(2).reshape(BATCH_SIZEV*(sqe800))
                data_out5 = data_out3.squeeze(2).reshape(BATCH_SIZEV,(sqe800)).permute(1,0)
        
                target[:,:,:] = y3[:,:sqe800,:] 
                t_x = Variable(data_in.to(device))
                t_x2 = Variable(data_in2.to(device))
                #t_y = Variable(data_out.to(device))
                t_y2 = Variable(data_out4.to(device))
                t_mask = Variable(data_mask.to(device))
                t_y3 = Variable(data_out5.to(device))
                trg = Variable(target.to(device))

                tmp = cnnmodel(t_x, t_x2)
                outputs = model(tmp, t_x, trg, 0.0)


                _, pred2 = torch.max(outputs[:,:,:].contiguous().view(-1,cls).data, 1)
                plabel = pred2.cpu().data
                #print(plabel.size())
                
                for tt in range(BATCH_SIZEV):
                    #if data_out4z[(tt+1)*(sqe-1)-1] == 1:
                    for s in range(sqe800):
                        if z[tt,s,0] ==1:
                            cf[data_out4[(tt*sqe800)+(s)], plabel[(tt*sqe800)+(s)]] +=1
                            total[data_out4[(tt*sqe800)+(s)]] +=1
                            #print(plabel[tt])
                            #print(data_out4[(tt+1)*sqe-1])
                            if plabel[(tt*sqe800)+(s)] == data_out4[(tt*sqe800)+(s)]:
                                acc[data_out4[(tt*sqe800)+(s)]] += 1
                                count += 1
               
        acc_t[epoch] =count/total.sum()
        print("Train EPOCH : ", epoch + 1, " ACC : ", count/total.sum(), "    LR: ", LR)
        print("ACC 0 wake : ", acc[0]/total[0], " ACC 1 stage 1 : ", acc[1]/total[1], " ACC 2 stage 2 : ", acc[2]/total[2], " ACC 3 stage 3 : ", acc[3]/total[3])
        print(total[0], "   ", total[1], "   ", total[2] , "   ", total[3])
        np.set_printoptions(precision=3,suppress=True)
        print("Train Confusion matrix : ")
        print(cf)





    if (epoch+1)%1 == 0:
        mse = np.zeros(BATCH_SIZEV2)
        pmse = np.zeros(BATCH_SIZEV2)
        count = 0.0
        acc = np.zeros(cls)
        total = np.zeros(cls)
        cf = np.zeros((cls,cls))

        sqe1200 = 1200
        for step, (x, y, y2, lens, z, time) in enumerate(val_loader):

            with torch.no_grad():


                y3 = y2

        
                #ii = 
                data_in = torch.FloatTensor(BATCH_SIZEV2, sqe1200*30, in_w)
                data_in2 = torch.FloatTensor(BATCH_SIZEV2, sqe1200, 5)
                data_out = torch.LongTensor(BATCH_SIZEV2, 1,1)
                data_out3 = torch.LongTensor(BATCH_SIZEV2, sqe1200, 1)
                data_mask = torch.LongTensor(BATCH_SIZEV2, sqe1200, 1)
                target = torch.FloatTensor(BATCH_SIZEV2, sqe1200, cls)
        
                data_in[:,:,:] = x[:,:sqe1200*30,:]
                data_in2[:,:,:] = time[:,:sqe1200,:5]

                data_out3[:,:,:] = y[:,:sqe1200,:]
                data_mask[:,:,:] = z[:,:sqe1200,:]
                
                data_out4 = data_out3.squeeze(2).reshape(BATCH_SIZEV2*(sqe1200))
                data_mask = data_mask.squeeze(2).reshape(BATCH_SIZEV2*(sqe1200))
                data_out5 = data_out3.squeeze(2).reshape(BATCH_SIZEV2,(sqe1200)).permute(1,0)
        
                target[:,:,:] = y3[:,:sqe1200,:] 
                t_x = Variable(data_in.to(device))
                t_x2 = Variable(data_in2.to(device))
                #t_y = Variable(data_out.to(device))
                t_y2 = Variable(data_out4.to(device))
                t_mask = Variable(data_mask.to(device))
                t_y3 = Variable(data_out5.to(device))
                trg = Variable(target.to(device))
                tmp = cnnmodel(t_x, t_x2)
                outputs = model(tmp, t_x,trg, 0.0)

                _, pred2 = torch.max(outputs[:,:,:].contiguous().view(-1,cls).data, 1)
                plabel = pred2.cpu().data
                for tt in range(BATCH_SIZEV2):

                    #if data_out4z[(tt+1)*(sqe-1)-1] == 1:
                    l = lens[tt]
                    for s in range(l):
                        if z[tt,s,0] ==1:
                            cf[data_out4[(tt*sqe1200)+(s)], plabel[(tt*sqe1200)+(s)]] +=1
                            total[data_out4[(tt*sqe1200)+(s)]] +=1
                            if plabel[(tt*sqe1200)+(s)] == data_out4[(tt*sqe1200)+(s)]:
                                acc[data_out4[(tt*sqe1200)+(s)]] += 1
                                count += 1




        acc_v[epoch] = count/total.sum()                   
        print("Val Mode EPOCH : ", epoch + 1, " ACCV : ", count/total.sum(), "    LR: ", LR)
        print("ACC 0 Vwake : ", acc[0]/total[0], " ACC 1 stage V1 : ", acc[1]/total[1], " ACC 2 stage V2 : ", acc[2]/total[2], " ACC 3 stage V3 : ", acc[3]/total[3])
        print(total[0], "   ", total[1], "   ", total[2] , "   ", total[3])



        np.set_printoptions(precision=3,suppress=True)
        print("Val Mode Confusion matrix : ")
        print(cf)

        #Saving model
        #SAVE_PATH_CNN = "/CNN_LSTM_Epoch_" + str(epoch + 1) + ".pkl"
        #SAVE_PATH_S2S = "/S2S_Epoch_" + str(epoch + 1) + ".pkl"
        #torch.save(cnnmodel.state_dict(), SAVE_PATH_CNN)
        #torch.save(model.state_dict(), SAVE_PATH_S2S)
        #print("Model saved!!")